In [1]:
import os
%pwd

'/home/atom/project/ENE_mlops_mlflow/research'

In [2]:
os.chdir("../")
%pwd

'/home/atom/project/ENE_mlops_mlflow'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    test_data_path : Path
    train_data_path : Path
    model_name : str
    alpha : float       #---------- from params.yaml
    l1_ratio : float    #--------|
    target_column : str #========== from schema.yaml

In [4]:
from src.mlProject.constant import *
from src.mlProject.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManger:

    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)    
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        # this below code will create artifact of those ^^ files
        create_directories([self.config.artifacts_root])



    def get_model_trainer_config(self)-> ModelTrainerConfig:
        # using these file cause we need them in "model_trainer_config"
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir, 
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

In [14]:
import os
import joblib
import pandas as pd
from sklearn.linear_model import ElasticNet
from src.mlProject import logger

In [23]:
class ModelTrainer:
    def __init__(self, config : ModelTrainerConfig):
        self.config = config

    def train(self):
        # taking data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1) # droping target column
        test_x = test_data.drop([self.config.target_column], axis=1)    # droping target column
        train_y = train_data[[self.config.target_column]]   # keeping target column
        test_y = test_data[[self.config.target_column]] # keeping target column

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x,train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [24]:
try:
    config = ConfigurationManger()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config= ModelTrainer(config = model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-05-20 17:20:45,622: INFO: common: yaml file :config/config.yaml is loaded successfully]
[2024-05-20 17:20:45,625: INFO: common: yaml file :params.yaml is loaded successfully]
[2024-05-20 17:20:45,629: INFO: common: yaml file :schema.yaml is loaded successfully]
[2024-05-20 17:20:45,630: INFO: common: created directories at artifacts]
[2024-05-20 17:20:45,631: INFO: common: created directories at artifacts/model_trainer]
